# Building the HAND model

In [ ]:
import subprocess
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
from rasterio import plot as rioplot

In [ ]:
# For simplicity, we use subprocess to run here, but we usually just run these commands
# in a terminal window.
fim_pipeline_help = "fim_pipeline.sh --help"
subprocess.run(fim_pipeline_help, shell=True)

## Running the FIM Pipeline

In [ ]:
fim_output_dir = 'fim_pipeline_outputs'
huc = '03100204'
fim_pipeline_cmd = f"fim_pipeline.sh -o -u {huc} -n {fim_output_dir} -jh 1 -jb 5"
print(fim_pipeline_cmd)
subprocess.run(fim_pipeline_cmd, shell=True)

### FIM Pipeline outputs

In [ ]:
from pathlib import Path
from itertools import islice

space =  '    '
branch = '│   '
tee =    '├── '
last =   '└── '


def tree(dir_path: Path, level: int=-1, limit_to_directories: bool=False,
         length_limit: int=1000):
    """Given a directory Path object print a visual tree structure"""
    dir_path = Path(dir_path) # accept string coerceable to Path
    files = 0
    directories = 0
    def inner(dir_path: Path, prefix: str='', level=-1):
        nonlocal files, directories
        if not level: 
            return # 0, stop iterating
        if limit_to_directories:
            contents = sorted([d for d in dir_path.iterdir() if d.is_dir()])
        else: 
            contents = sorted(list(dir_path.iterdir()))
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + path.name
                directories += 1
                extension = branch if pointer == tee else space 
                yield from inner(path, prefix=prefix+extension, level=level-1)
            elif not limit_to_directories:
                yield prefix + pointer + path.name
                files += 1
    print(dir_path.name)
    iterator = inner(dir_path, level=level)
    for line in islice(iterator, length_limit):
        print(line)
    if next(iterator, None):
        print(f'... length_limit, {length_limit}, reached, counted:')
    print(f'\n{directories} directories' + (f', {files} files' if files else ''))

In [ ]:
tree(Path(Path.home().parent, 'outputs', fim_output_dir, huc), level=2)

In [ ]:
tree(Path(Path.home().parent, 'outputs', fim_output_dir, huc, 'branches', '1691000003'))

# Producing an inundation map

In [ ]:
inundation_raster = os.path.join('/outputs', fim_output_dir, 'fim_raster.tif')
flow_file_csv = "/data/inundation_review/inundation_nwm_recurr/nwm_recurr_flow_data/nwm21_17C_recurr_50_0_cms.csv"
fim_pipeline_cmd = f"python /foss_fim/tools/inundate_mosaic_wrapper.py -y /outputs/{fim_output_dir} -u {huc} -i {inundation_raster} -f {flow_file_csv} -w 5"
print(fim_pipeline_cmd)
subprocess.run(fim_pipeline_cmd, shell=True)

## Plotting FIM

### Show full FIM raster

In [ ]:
with rasterio.open(inundation_raster) as fim_rast:
    fim_nodata = fim_rast.profile['nodata']
    fim_extent = rioplot.plotting_extent(fim_rast)
    fim_transform = fim_rast.transform
    fim_crs = fim_rast.crs
    fim = fim_rast.read(1).astype(float)
    # Set nodata to nan
    fim[np.where(fim == fim_nodata)] = np.nan
    # Create binary array where 1 = wet and 0 = dry
    fim[np.where(fim > 0)] = 1
    fim[np.where(fim < 0)] = np.nan

# Plot using matplotlib
fig = plt.figure()
ax = fig.subplots(1,1)

im = ax.imshow(fim, cmap='bwr', extent=fim_extent, interpolation='none', alpha=1)

ax.set_xticks([])
ax.set_yticks([])
ax.set_title("NWM Retrospective 50-yr FIM")
fig.tight_layout()
plt.show()

### Comparing base HAND with FIM v4.X

In [ ]:
# Load branch 0 raster
branch_0_inundation_raster = inundation_raster.replace('.tif', f'_{huc}_0.tif')
with rasterio.open(branch_0_inundation_raster) as b0_rast:
    branch0_nodata = b0_rast.profile['nodata']
    branch0_extent = rioplot.plotting_extent(b0_rast)
    branch0_transform = b0_rast.transform
    branch0_crs = b0_rast.crs
    branch0 = b0_rast.read(1).astype(float)
    # Set nodata to nan
    branch0[np.where(branch0 == branch0_nodata)] = np.nan
    # Create binary array where 1 = wet and 0 = dry
    branch0[np.where(branch0 > 0)] = 1
    branch0[np.where(branch0 < 0)] = np.nan
branch_0_inundation_raster

In [ ]:
# Reqesting ESRI basemap
fim_crs = '5070'
# X/Y bounds of the maps
x = (1346850, 1352450)
y = (628000, 633600)
# Grabbing a basemap from Arcgis Online
esri_url = f"https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/export?bbox={x[0]}%2C{y[0]}%2C{x[1]}%2C{y[1]}&bboxSR={fim_crs}&layers=0&size=&imageSR={fim_crs}&transparent=true&dpi=200&f=image"
esri_aerial = Image.open(requests.get(esri_url, stream=True).raw).convert('RGBA')
esri_url

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.subplots(1,3)

# Subplot 1
ax[0].set_title("Base HAND i.e. Branch Zero")
im = ax[0].imshow(esri_aerial, extent=(x[0],x[1],y[0],y[1]))
im = ax[0].imshow(
    branch0,
    cmap='bwr',
    extent=branch0_extent,
    interpolation='none',
    alpha=0.7
)

 # Subplot 2
ax[1].set_title("Composited FIM v4.x with Generalized Mainstems (GMS)")
im = ax[1].imshow(esri_aerial, extent=(x[0],x[1],y[0],y[1]))
im = ax[1].imshow(fim, cmap='bwr', extent=fim_extent, interpolation='none', alpha=0.7)

# Subplot 3
ax[2].set_title("Highlighting the Differences")
im = ax[2].imshow(fim, cmap='bwr_r', extent=fim_extent, interpolation='none', alpha=0.4)
im = ax[2].imshow(branch0, cmap='bwr', extent=branch0_extent, interpolation='none', alpha=0.4)

# Set the bounds of all axes to be the same
for axis in ax:
    axis.set_xbound([x[0],x[1]])
    axis.set_ybound([y[0],y[1]])
    axis.set_xticks([])
    axis.set_yticks([])
fig.tight_layout()
plt.show()

# Evaluating FIM performance

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch

In [ ]:
fim_pipeline_cmd = f"python /foss_fim/tools/synthesize_test_cases.py -c DEV -e GMS -v {fim_output_dir} -jh 1 -jb 5 -o -b nws -m /outputs/{fim_output_dir}/eval_metrics.csv"
print(fim_pipeline_cmd)
subprocess.run(fim_pipeline_cmd, shell=True)

## Plotting the agreement map

In [ ]:
agreement_map_tif = f"/data/test_cases/nws_test_cases/{huc}_nws/testing_versions/{fim_output_dir}/major/litf1_b0m_agreement.tif"

In [ ]:
# Load in the agreement raster
with rasterio.open(agreement_map_tif) as agree:
    transform = agree.transform
    agree_grid = agree.read(1)
    agree_extent = rioplot.plotting_extent(agree)
    agree_grid = np.ma.masked_where(agree_grid == 10, agree_grid)

In [ ]:
# Custom colormap for the agreement raster
cdict = {'red':  ((0.0, 0.0, 0.0),     # 0: True Negative
                  (0.25, 0.91, 0.91),  # 1: False Negative
                  (0.5, 0.93,0.93),    # 2: False Positive
                  (0.75, 0.43, 0.43),  # 3: True Positive
                  (1.0, 0.77, 0.77)),  # 4: Masked Area
         'green':((0.0, 0.0, 0.0),
                  (0.25, 0.73, 0.73),
                  (0.5, 0.04, 0.04),
                  (0.75, 0.61, 0.61),
                  (1.0, 0.77, 0.77)),
          'blue':((0.0, 0.0, 0.0),
                  (0.25, 0.0, 0.0),
                  (0.5, 0.05, 0.05),
                  (0.75, 0.88, 0.88),
                  (1.0, 0.77, 0.77))
          }
agree_colormap = LinearSegmentedColormap('agree', cdict)
norm=plt.Normalize(0,4)

In [ ]:
# Plotting code
fig, ax = plt.subplots(figsize=(12, 8))

ax.imshow(agree_grid, cmap=agree_colormap, norm=norm, extent=agree_extent, interpolation='none', alpha=1)
ax.set_xticks([])
ax.set_yticks([])
# Legend
TP_patch = Patch(color='#2c7bb6', linewidth=0.5, label='True Positive')
TN_patch = Patch(color='#000000', linewidth=0.5, label='True Negative')
FN_patch = Patch(color='#fdae61', linewidth=0.5, label='False Negative')
FP_patch = Patch(color='#d7191c', linewidth=0.5, label='False Positive')
ax.legend(loc='upper left', ncol=1, handles=[TP_patch, FN_patch, FP_patch, TN_patch], fontsize=12)
ax.set_title('LITF1 Benchmark Agreement')
plt.tight_layout()
plt.show()